In [ ]:
print hit4hmm2hsp.objects.count()
# s35s = classification.objects.all()
# s35 = s35s[12]

In [1]:
import matplotlib
matplotlib.use("Agg")
from matplotlib import pyplot as plt
%matplotlib inline

import os
os.environ["SEQlib"] = '/data/sequence-data'
os.environ["SEQlib"] = '$repos/cathdb'
import re

p_hmmlen = re.compile('LENG  (\d+)\n')

import numpy as np
from tst.domutil.util import *
from time import time

/export/people/zcbtfge/Documents/repos/test_envs/django_CATH/local/lib/python2.7/site-packages/subprocess32.py:472: RuntimeWarning: The _posixsubprocess module is not being used. Child process reliability may suffer if your program uses threads.
  "program uses threads.", RuntimeWarning)
/export/people/zcbtfge/Documents/repos/test_envs/django_CATH/local/lib/python2.7/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [3]:
from time import time 
import cPickle as pk
t0 = time()
# pk.dump(D,open('CATHS100.pk','wb'))
D = pk.load(open('data/CATHS100.pk','rb'))
# np.ndarray?
print time() - t0

32.472258091


In [ ]:
# hmms = HMMprofile.objects.all()[:200]
oall_hmms = HMMprofile.objects.defer('text').all().annotate(hcount = Count('hits')).order_by('-hcount')

In [ ]:
lgs = [int(p_hmmlen.findall(hmm.text)[0]) for hmm in oall_hmms]

In [ ]:
hmms = HMMprofile.objects.all().order_by
xs = []
ys = []
for hmm in hmms:
    hmm_text = hmm.text
    y = hmm.hits.count()
    if y:
        xs += [int(p_hmmlen.findall(hmm_text)[0])]
        ys += [ y ]
    
kwargs = {}
    
plt.close()
siz = [450,400]
DPI=100.
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
ax1.set_yscale('log')
points = ax1.scatter(xs,ys, 
    # colors,
    edgecolor = 'b',facecolor = 'b',
    # edgecolor = 'r',facecolor = 'r',
    marker ='o',
    # c = colors,
     s =100,
     alpha = 0.15,
     # **kwargs
     )


In [ ]:
hmms = oall_hmms
# xs = []
# ys = []
# for hmm in hmms:
#     hmm_text = hmm.text
#     y = hmm.hits.count()
#     if y:
#         xs += [int(p_hmmlen.findall(hmm_text)[0])]
#         ys += [ y ]
xdata  = hmms.values_list('hcount',flat = True)
# xs = [x for x in xdata if x>10]

In [ ]:
kwargs = {}
plt.close()
siz = [450,400]
DPI=100.
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
# ax1.set_yscale('log')
# ax1.set_xscale('log')
xs = np.maximum(0,np.log(1+np.array(xdata)))
bins = ax1.hist(xs,300, 
    # colors,
    edgecolor = 'b',facecolor = 'b',
    # edgecolor = 'r',facecolor = 'r',
#     marker ='o',
    # c = colors,
#      s =100,
#      alpha = 0.15,
     # **kwargs
     )

In [10]:
from tst.domutil.util import *
# its = D.iteritems()
its = using_tocoo_izip(D)

xs = []
ys = []
zs = []
sfbools = [];

c = counter(range(6563388),per = 100000)

for xi,yi,inter_ct in its:
    c.count()
#     xct = 
    x = hcounts[xi]
    y = hcounts[yi]

    xs.append( x )
    ys.append( y )
    zs.append(inter_ct)
    sfbools.append(sfs[xi]==sfs[yi])
#     normz = log10()
# print next(its)
# print next(its)

# ys 

0 of 6563388


IndexError: list index out of range

In [ ]:
xss = np.log10(np.array(xs)+1)
yss = np.log10(np.array(ys)+1)
zss = np.log10(np.array(zs)+1)

In [ ]:
normz = (xss+yss)/2.-zss

In [ ]:
import sys
print >>sys.__stdout__, next(D.iteritems())

In [ ]:
from scipy.sparse import *
import numpy as np

Dnorm = coo_matrix( (normz ,zip(*[k for k,v in D.items()]) ),shape = D.shape, dtype = 'float')




In [ ]:
c = counter([],INF=1)

cx = X   
# for i,j,v in itertools.izip(cx.row, cx.col, cx.data):
#     c.count()
#     print (i,j,v)
for x,y,v in using_tocoo_izip(Dnorm):
    c.count()
    print>>sys.__stdout__, x,y,v
    if c.i == 10:
        break
    

In [ ]:
import cPickle as pk
t0 = time()
# pdata = [xs,ys,zs,sfbools]
pk.dump(Dnorm,open('data/Dnorm','wb'))
# pdata  =  pk.load(open('pdata','rb'))
# (xs,ys,zs,sfbools) = pdata;

print>>sys.__stdout__, time()-t0
print "finished"

In [ ]:
# kde([[1,2],[12,4],[3,6]])
colors = kde([xss,yss,zss])

In [ ]:
import matplotlib
%matplotlib inline
from matplotlib import cm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
kwargs = {
    'projection':'3d',
}
plt.close()
siz = [450,400]
DPI=100.
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
# idx = range(1000)
N = 1000
xss = np.array(xs)
yss = np.array(ys) 
zss = np.array(zs)

idx = sfbools
xss = np.delete(xss,idx)
yss = np.delete(yss,idx)
zss = np.delete(zss,idx)

xss= np.array(xss[:N])
yss = np.array(yss[:N])
zss = np.array(zss[:N])

xss = np.log10(xss+1)
yss = np.log10(yss+1)
zss = np.log10(zss+1)

xyz = np.vstack([xss,yss,zss])
kde = stats.gaussian_kde(xyz)
# ax1.set_zlim([0,2000])

ax1.scatter(xss,yss,zss,'.',3, c = kde([xss,yss,zss]))

X = np.linspace(xss.min(), xss.max(),30)
Y = np.linspace(yss.min(), yss.max(),30)
# Z = np.linspace(zss.min(), zss.max(),30)
# Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
Z = (X + Y) / 2 - 2
surf = ax1.plot_surface(X, Y, Z, cmap=cm.coolwarm, alpha = 0.5, 
                       linewidth=0, antialiased=False)

In [ ]:
anim_0 = anim
HTML(anim_0.to_html5_video())

In [ ]:
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)

def init():
#     ax1 = fig.add_subplot(111,**kwargs)
    pts = ax1.scatter(xss,yss,zss,'.',3, c = kde([xss,yss,zss]))# for angle in range(0, 360):
    surf = ax1.plot_surface(X, Y, Z, cmap=cm.coolwarm, alpha = 0.5, 
                       linewidth=0, antialiased=False)
    return (fig,)


def animate(i):
#     angle = i*0.5
#     pts = ax1.scatter(xss,yss,zss,'.',3, c = kde([xss,yss,zss]))# for angle in range(0, 360):
#     ax1.view_init(30, angle
    ax1.view_init(elev=10., azim=i*10)
#     plt.draw()
#     plt.pause(.001)
    return (fig,)
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=360, interval=20, blit=True);
HTML(anim.to_html5_video())

In [ ]:
animation.FuncAnimation?

In [ ]:
anim_2 = anim
HTML(anim_2.to_html5_video())

In [ ]:

from matplotlib import animation, rc
from IPython.display import HTML

fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
ax1.scatter(xss,yss,zss,'.',3, c = kde([xss,yss,zss]))# for angle in range(0, 360):

for angle in range(0, 360):
    ax1.view_init(30, angle)
    plt.draw()
    plt.pause(.001)
angle = 110
ax1.view_init(30, angle)
# plt.draw()
# plt.show()
# plt.pause(.001)
# print 
# ax1.set_yscale('log')
# ax1.set_xscale('log')
# ax1.xaxis.set_scale('log')
# ax1.yaxis.set_scale('log')

In [ ]:
for angle in range(0, 360):
    ax1.view_init(30, angle)
    plt.draw()
#     plt.show()
    plt.pause(.001)

In [ ]:
import cPickle as pk
t0 = time()
# pdata = [xs,ys,zs,sfbools]
# pk.dump(pdata,open('pdata','wb'))
pdata  =  pk.load(open('pdata','rb'))
(xs,ys,zs,sfbools) = pdata;

print time()-t0
print "finished"

In [ ]:
x = np.array(xs)
y = np.array(ys)
z = np.array(zs)
idx = z>10

In [ ]:
print sum(idx)

In [ ]:
x = np.log(x[idx]+1)
y = np.log(y[idx]+1)
z = np.log(z[idx]+1)

In [ ]:
# sum(sfbools)
np.where(sfbools) == 0

In [ ]:

import sys  
reload(sys)  
sys.setdefaultencoding('utf8')


In [ ]:
import numpy as np
from scipy import stats
from mayavi import mlab


mlab.init_notebook(width = 800,height = 800)

N = 1000

x = np.array(xs)
y = np.array(ys)
z = np.array(zs)

idx = sfbools
x = np.delete(x,idx)
y = np.delete(y,idx)
z = np.delete(z,idx)
x = np.array(x[:N])
y = np.array(y[:N])
z = np.array(z[:N])
x = np.log(x+1)
y = np.log(y+1)
z = np.log(z+1)

xyz = np.vstack([x,y,z])
kde = stats.gaussian_kde(xyz)

figure = mlab.figure('DensityPlot', size=(1280, 720))
figure.scene.disable_render = True

density = kde(xyz)
pts = mlab.points3d(x, y, z, density, scale_mode='none', scale_factor=0.07) 
mask = pts.glyph.mask_points
mask.maximum_number_of_points = x.size
mask.on_ratio = 1
# pts.glyph.mask_input_points = True
figure.scene.disable_render = False 
# mlab.axes()
# mlab.show()
# mlab.outline()
pts

In [ ]:
# Evaluate kde on a grid
xmin, ymin, zmin = x.min(), y.min(), z.min()
xmax, ymax, zmax = x.max(), y.max(), z.max()
xi, yi, zi = np.mgrid[xmin:xmax:30j, ymin:ymax:30j, zmin:zmax:30j]
coords = np.vstack([item.ravel() for item in [xi, yi, zi]]) 
density = kde(coords).reshape(xi.shape)

# Plot scatter with mayavi
figure = mlab.figure('DensityPlot')

grid = mlab.pipeline.scalar_field(xi, yi, zi, density)
MIN = density.min()
MAX = density.max()
mlab.pipeline.volume(grid, vmin=MIN, vmax=MIN + .5*(MAX-MIN))

In [ ]:
from mayavi import mlab
mlab.init_notebook()
s = mlab.test_plot3d()
s


In [ ]:
import numpy as np
from scipy import stats
from mayavi import mlab

# mu, sigma = 0, 0.1 
# x = 10*np.random.normal(mu, sigma, 500)
# y = 10*np.random.normal(mu, sigma, 500)
# z = 10*np.random.normal(mu, sigma, 500)

xyz = np.vstack([x,y,z])
kde = stats.gaussian_kde(xyz)
density = kde(xyz)



In [ ]:
figure = mlab.figure('DensityPlot',size = [600,600])
figure.scene.disable_render = True

pts = mlab.points3d(x, y, z, density, scale_mode='none', scale_factor=0.07) 
mask = pts.glyph.mask_points
mask.maximum_number_of_points = x.size
mask.on_ratio = 1
# pts.glyph.mask_input_points = True
figure.scene.disable_render = False 
mlab.axes()
# mlab.show()
# mlab.outline()
pts

In [ ]:
del figure
# import gc 
# gc.collect()

In [ ]:
from scipy.spatial.distance import *
# from sklearn.metrics.pairwise import pairwise_distances as pdist
# def ISS(us,vs):
# #     u.hits
#     D = []
#     for ui,u in enumerate(us):
#         for v in vs[ui:]:
#             D.append(0)
# #             D.append(u.)
# #             D.append(0)
#     D = squareform(D)
# #     print D.dims
#     return D


def ISS(u,v):
    u = u[0]; v = v[0]
#     print u[0]
#     u_hitids = u.hits.values_list("id",flat = True)
#     v_hitids = v.hits.values_list("id",flat = True)
    d = float((u.hits.all() & v.hits.all() ).count())/ (u.hits.count() * v.hits.count() + 1)
#     d = u.hits.count()
    return d
def ISS(u,v):
#     c.count()
    u = u[0]; v = v[0]
    ulen = len(u)
    vlen = len(v)
#     if ulen <= vlen:
#         inter_count = sum(u_curr in v for u_curr in u )
#     else:
#         inter_count = sum(u_curr in u for u_curr in v )
    inter_count = len(set(u) & set(v))
    d = 1 - float(2*inter_count)/(ulen+vlen+1)

    return d

In [ ]:
X = np.expand_dims(
    np.array(
    [list(
        hmm.hits.values_list("id", flat = True)
    )
     for hmm in hmms]
), axis = 1)

t0 = time()
c = counter(range(X.size**2/2))

dm = pdist( X = X,metric =  ISS)
# D = squareform(dm)
print time()-t0
# X[0]

In [ ]:
hmms = HMMprofile.objects.all()[:200]
X = np.expand_dims(
    np.array(
    [list(
        hmm.hits.values_list("id", flat = True)
    )
     for hmm in hmms]
), axis = 1)



In [ ]:
t0 = time()
c = counter(range(X.size**2/2))

dm = pdist( X = X,metric =  ISS)
# D = squareform(dm)
print time()-t0

In [ ]:
t0 = time()
c = counter(range(len(X)**2/2),per = 1E3)
from multiprocessing import *


def batch(X,batch_size = 5 ):
    lst = []
    i = 0
    for x in X:
        i += 1
        lst += [x]
        if not i % batch_size:
            yield lst
            lst = []
# print next(g)

def queue_gen(X):
    i = 0
    for ui,u in enumerate(X):
        for v in X[ui+1:]:
            i += 1
            yield [u,v]

g  = batch(queue_gen(X),5)

    
def ISS0(x):
#     c.count()
    return(ISS(*x))
def ISS1(xs):
#     c.count()
    return([ISS(*x) for x in xs] )

# if 1:
N = 10
t0 = time()
for i in range(N):
    if __name__ == '__main__':
        p = Pool(5)
        dm = (p.map_async(ISS0, (queue_gen(X))))
        dm = dm.get()
    #     sum(p.map(ISS1, batch(queue_gen(X)), 200),[] )
        p.close()
print time()-t0 
t0 = time()
for i in range(N):
    if __name__ == '__main__':
        p = Pool(5)
#         dm1 = (p.map(ISS0, (queue_gen(X)), chunksize = 5))
        dm1 = (p.map(ISS0, (queue_gen(X)), ))
    #     sum(p.map(ISS1, batch(queue_gen(X)), 200),[] )
        p.close()
print time()-t0 
t0 = time()
for i in range(N):
    if __name__ == '__main__':
        dm2 = pdist(X,ISS)
print time()-t0 

In [ ]:
for i in range(10):
    if __name__ == '__main__':
        p = Pool(7)
#         r = (p.map(ISS0, (queue_gen(X))))
        sum(p.map(ISS1, batch(queue_gen(X)), 100),[] )
        p.close()
print time()-t0 

In [ ]:
all_hmms = HMMprofile.objects.annotate(hcount = Count("hits"))

In [ ]:
# hmms = all_hmms.exclude(hcount__lte=0)
print hmms.count()

In [ ]:
sDB = seqDB.objects.get(name = 'CATH')
sDB.sequence_set.count()
seq = sDB.sequence_set.first()
# seq.HMMprofile
# seq.hmmprofile
# dir(seq)
seq.hmmprofile_set.count()

In [ ]:
ids = HMMprofile.objects.values_list('id',flat = True)
print max(ids)- min(ids)
print min(ids)
print len(ids)


In [ ]:
a = np.ndarray(shape = [21086,21086],dtype = 'int')
# np.ndarray?
# a[0]

In [ ]:
c = counter(range(sDB.sequence_set.count()))
seqset_all = sDB.sequence_set.all()

import itertools as it
def pair_gen(seqset_all):
    c = counter(range(seqset_all.count()))
    batches = batch_qs( seqset_all, 500)
    for seqset in batches:
        
#             hmmids = seq.values_list("hmmprofile",flat =True)
        for seq in seqset:
            c.count()
            hmmids = seq.hmmprofile_set.values_list("id",flat = True)
            pairs = it.combinations(sorted(hmmids),2)
            for p in pairs:
                yield p

                
from collections import Counter
import multiprocessing as mp



t0 = time()    
g = pair_gen(seqset_all.all())
counts = Counter(g)
print time()-t0
# set(g)

In [ ]:
from scipy.sparse import *
# D = csr_matrix(zip(*counts.keys()),counts.values())
# D = dok_matrix((21086,21086),dtype = np.int)

t0 = time()    
c=counter(counts,per=10000)
for (x,y),v in counts.iteritems():
    c.count()
    D[x-1,y-1] = v
print time()-t0

In [ ]:
cD = D.to_cs

In [ ]:
g = pair_gen(seqset_all.all()[:2000])

for i in range(10):
    if __name__ == '__main__':
        p = Pool(7)
#         r = (p.map(ISS0, (queue_gen(X))))
        sum(p.map(ISS1, batch(queue_gen(X)), 100),[] )
        p.close()

for x,y in g:
#     print x,y
    a[x-1,y-1] += 1
    

In [ ]:
# from itertools
import itertools as it
print hmmids.count()

pairs = it.combinations(sorted(hmmids[:20]),2)
for p in pairs:
    yield p

In [ ]:
X = np.expand_dims(
    np.array(
    [list(
        hmm.hits.values_list("id", flat = True)
    )
     for hmm in hmms]
), axis = 1)

In [ ]:
us = X
vs = X
dm = []

for ui,u in enumerate(us):
    for v in vs[ui:]:
        dm.append( ISS(u,v) )
D = squareform(dm)

In [ ]:
# dm.size

# None in []

In [ ]:
D.shape

In [ ]:
us = X
vs = X
# x =
# D = []
# for ui,u in enumerate(us):
#     for v in vs[ui:]:
#         D.append(0)
# D = squareform(D)
# # print D.dims

# dm = pdist( X, ISS)
# D = squareform(dm)
plt.close()
siz = [450,400]
DPI=100.
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
imDmat = ax1.imshow(D)
plt.colorbar(imDmat)
# X.shape
# dm = cdist( X.T,X.T, ISS)
# np.array(list(hmms))

In [ ]:
plt.close()
siz = [450,400]
DPI=100.
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
imdata = np.log(D)
imDmat = ax1.imshow(imdata)
plt.colorbar(imDmat)

In [ ]:
hmm1.hit4hmm2hsp_set

In [ ]:
# hmm1.hits.count()
# hmm2.hits.count()

In [ ]:
hmm1.hits.all().count()

In [1]:
# os.path.isfile(full("$PDBlib/1a41A01"))
hmm1.hits.all()

NameError: name 'hmm1' is not defined

In [ ]:
inter_hitseq = hmm1.hits.all() & hmm2.hits.all()
hits1 = hmm1.hit4hmm2hsp_set.filter(target__in = inter_hitseq)
hits2 = hmm2.hit4hmm2hsp_set.filter(target__in = inter_hitseq)
fmt = "%s,\t domain:%s\t length:%s,\t hitnum:%d"
print "intersection count is",inter_hitseq.count()
print  fmt %(hmm1,hmm1.cath_node.domain, p_hmmlen.findall(hmm1.text)[0], hmm1.hits.count() )
print fmt % (hmm2,hmm2.cath_node.domain, p_hmmlen.findall(hmm2.text)[0], hmm2.hits.count() )
for i,j in zip(hits1,hits2):
    try:
        node =  domain.objects.get(domain_id = i.target.acc).classification
    except:
        node = ''
    print i.target.acc,i.bitscore,
    print j.target.acc,j.bitscore
#     print node
    print i.start,j.start
    print i.end,j.end

In [ ]:
hmm1.cath_node.parent

In [ ]:
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return np.unravel_index(indices, ary.shape)
# D[largest_indices(D,10)]
D = squareform(dm)
tD = np.triu(D[-2000:,-2000:])

lst = zip(*largest_indices(tD,20000))
len(D)

In [ ]:
# hmms.count()
D[-1,-2]

In [ ]:
c = counter(lst,per = 1)
i = 0
for idxx,idxy in lst:
#     print
    hmm1 = hmms[idxx]
    hmm2 = hmms[idxy]
#     if hmm1.cath_node.parent == hmm2.cath_node.parent:
    cnode1 = hmm1.cath_node
    cnode2 = hmm2.cath_node
    if cnode1.Class == cnode2.Class:
        continue
    c.count()   
    
    if c.i >= 2:
        i+=1
        print hmm1
        print hmm2
        print D[idxx,idxy]
        print hmm1.hits.count() + hmm2.hits.count()
        print "http://xhits.cathdb.info/crosshits.php?sf2=%s&sf1=%s&db_source=Crosshits_v4_1_0"%(cnode1.superfamily(),cnode2.superfamily())
        if i == 2:
            break

In [ ]:
for hmm in [hmm1,hmm2]:
    print hmm.cath_node.domain
    print '\n'.join(hmm.text.splitlines()[:10])
    print 
# print '\n'.join(hmm2.text.splitlines()[:10])

In [ ]:
xs = []
ys = []
for hmm in hmms:
    hmm_text = hmm.text
    y = hmm.hits.count()
    if y:
        xs += [int(p_hmmlen.findall(hmm_text)[0])]
        ys += [ y ]
    
kwargs = {}
    
plt.close()
siz = [450,400]
DPI=100.
fig = plt.figure(figsize = [x/float(DPI) for x in siz], dpi=DPI)
ax1 = fig.add_subplot(111,**kwargs)
ax1.set_yscale('log')
points = ax1.scatter(xs,ys, 
    # colors,
    edgecolor = 'b',facecolor = 'b',
    # edgecolor = 'r',facecolor = 'r',
    marker ='o',
    # c = colors,
     s =100,
     alpha = 0.15,
     # **kwargs
     )


In [ ]:
import subprocess
tmpdir = "/tmp/prc"
prcfmt = "http://update.cathdb.info/api/data/hmm/%s.prc"
dom  = hmm1.cath_node.domain
subprocess.check_call(["wget",prcfmt%dom,"-O","%s/%s.prc"%(tmpdir,dom)])
dom =  hmm2.cath_node.domain
subprocess.check_call(["wget",prcfmt%dom,"-O","%s/%s.prc"%(tmpdir,dom)])

In [ ]:
hmms = HMMprofile.objects.all()[:100]

In [ ]:
seqs  = sequence.objects.values_list('acc').annotate(ct= Count('acc'))

In [ ]:
seqdup = seqs.filter(ct__gt=1)
seqdup = sequence.objects.filter(acc__in=[x for x,y in seqdup])
# print seqdup.count()

In [ ]:
seqdup.delete()
# print seqdup.filter(hmmprofile__isnull=True).count()
# ss = sequence.objects.filter(acc=seqdup[0][0])
# ss.filter(hmmprofile__isnull=True).count()
# for s in ss:
#     print s.hmmprofile_set.count()

In [ ]:
# from tst.domutil.util import *
# from tst.utils_db import *

# # hmm = s35.hmmprofile_set.first()
# hmms = HMMprofile.objects.all()
# hmm = hmms[6]
# hmm.cath_node

# # print hmm.text

# seqDB_curr = seqDB.objects.get(name = 'CATH')
# seqDB_file = "$SEQlib/cath-domain-seqs-S100.fa"

# q_hits = hmmsearch(hmm,
# seqDB_curr = seqDB_curr,
# seqDB_file = seqDB_file,
# tmpdir = "/tmp/tmp2"                
# )

# oldhits = hmm.hit4hmm2hsp_set.all()
# for hit in q_hits:
#     hsp = hit[0] ### Assume only one dom per hit
#     jdict = hsp2jdict(hsp, query = hmm )
#     jdict["target_id"] = sequence.objects.get( acc = jdict["target_id"]).id

#     if not oldhits.filter(**jdict).exists():
#         hit_db = hit4hmm2hsp(**jdict)
#         hit_db.save()

# # print hmm

In [ ]:
print sequence.objects.count()
%load_ext autoreload
%autoreload 0
%autoreload 1
%autoreload 2
%autoreload 3


import os
os.environ['SEQlib'] = '/media/DATA/Documents/repos/cathdb/'


In [ ]:
p_cathheader = re.compile('\|([0-9,a-z,A-Z]*)\/')
with open(seqdb_file,'r') as f:
    buf = f.read() 
m  =p_cathheader.findall(buf)
# m  =p_cathheader.search(buf)
# next(m)
# m.groups()[0]
sum(len(x) for x in m)/float(len(m))
# dir(m)
# m.string
# print m.groupdict()
# print len(m)
# print len(set(m))

In [ ]:
from tst.utils_db import *
from Bio.SeqIO import FastaIO
debug = 1


behave = "import sequence database"
seqdb_file = full("$SEQlib/cath-domain-seqs-S100.fa")
# seqDB_curr = verify_exist_entry( seqDB, {'name' : 'test_CATH' })
# seqDB_curr = verify_exist_entry( {'name' : 'test_CATH',
#                                  'version': 'test'}, seqDB)
seqDB_curr = seqDB.objects.get(name='CATH')

from time import time
t0 = time()
useFisrt = 1
failcount = 0
lst = {
 '3hf9c00',
 '3hf9e00',
 '3hf9g00',
 '3hf9h00',
 '3hf9j00',
 '3hf9l00',
 '3hf9n00',
 '3hf9p00',
 '3hf9r00',
 '3hf9t00',
 '3hf9x00',
 '3hf9z00',
 '3lo3a00',
 '3lo3b00',
 '4il6z00'}
with open( seqdb_file ) as handle:
    iterator = FastaIO.SimpleFastaParser(handle)
    c = counter(iterator, per=1000)
with open( seqdb_file ) as handle:
    iterator = FastaIO.SimpleFastaParser(handle)

    with transaction.atomic():
        for values in iterator:
            try:
#             if 1:
                header,seq = values
                if c.i == 0:
                    (h_parser, _) = seqheader_guess_parser(header)
                    print "first entry pointing to %s" % seqDB_curr
                    print "h_parser is",h_parser
                    
                jdict = h_parser( header )
                jdict["length"] = len(seq)
                
                if useFisrt:
                    jdict["seqDB"] = seqDB_curr
                else:
                    jdict["seqDB"] = seqDB.objects.get(**jdict["seqDB"])
#                 worker()
#                 if jdict['acc']=='3hf9c00':
                if jdict['acc'] in lst:
                    print "im here"
                    local_worker(jdict)
#             try:
                pass
            except Exception as e:
                if debug: print e,header
                failcount += 1
            c.count()
        
#         print(values)
print "finshed %s from %s" % (behave, seqdb_file)

print '%d instances of %d failed' % (failcount,c.imax)
print('Ended after %.4f sec' % (time()-t0))  # len(lst)d
failrate = failcount/float(c.imax) 
assert failrate < 0.1, 'fail rate is too high: expected < 10%%, actual: %2.2f%%' % (failrate*100)

In [ ]:
def local_worker(jdict):
    seqobjs = seqDB_curr.sequence_set.filter(**jdict)
    if seqobjs.exists():
        print "passed"
        pass
    else:
        seqobj = sequence(**jdict)
        seqobj.save()
        print "inserted"

In [ ]:
# print jdict 
jdict['acc'] = '3hf9c00'
seqobjs = seqDB_curr.sequence_set.filter(acc__exact = '3hf9t00')
print seqobjs[0]


In [ ]:
acclst0 = set(m)
acclst = set(seqDB_curr.sequence_set.values_list("acc",flat = True))
print acclst0-acclst

In [ ]:
sequence.objects.count()